## Financial Market Analytics Project

In [ ]:
# All imports
import pandas as pd
import numpy as np
from scipy.stats import skew
from datetime import datetime, timedelta
import plotly.express as px
import plotly.io as pio


In [35]:
# Read the data, skipping the first row and using the second row as headers
# Read the data, skipping the first row and using the second row as headers
df_sp_m_new = pd.read_excel("../Datasets/SPX500.xlsx", sheet_name="Price", engine="openpyxl", skiprows=[0], header=0)
#df_stoxx_m_new = pd.read_excel("../Datasets/Stoxx600.xlsm", sheet_name="Price", engine="openpyxl", skiprows=[0], header=0)

df_sp_d_new = pd.read_excel("../Datasets/SPX500.xlsx", sheet_name="Price daily", engine="openpyxl", skiprows=[0], header=0)
#df_stoxx_d_new = pd.read_excel("../Datasets/Stoxx600.xlsm", sheet_name="Price daily", engine="openpyxl", skiprows=[0], header=0)


# Flatten the MultiIndex columns created by read_excel with header=0 and skiprows=[0]
df_sp_m_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sp_m_new.columns]
#df_stoxx_m_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_stoxx_m_new.columns]
df_sp_d_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sp_d_new.columns]
#df_stoxx_d_new.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_stoxx_d_new.columns]


print("Nuovo df_sp_m head:")
display(df_sp_m_new.head())

Nuovo df_sp_m head:


,data,3M Co,Abbott Laboratories,ACME-Cleveland Corp,Advanced Micro Devices Inc,AEP Texas Inc,Aeroquip-Vickers Inc,AIG Life Holdings Inc,Air Products & Chemicals Inc,Alberto-Culver Co,...,Cboe Global Markets Inc.1,United Continental Holdings Inc.1,WellCare Health Plans Inc,Advanced Micro Devices Inc.2,Fortinet Inc,Linde PLC,Rollins Inc,Jack Henry & Associates Inc,Keysight Technologies Inc,Regency Centers Corp.1
0,1990-12-31,20.629,5.0348,4.750,2.4375,22.0000,18.250,7.6875,12.6542,9.5408,...,NaN,NaN,NaN,NaN,NaN,NaN,1.2437,0.1250,NaN,NaN
1,1991-01-31,20.419,4.8810,5.375,3.5625,22.0000,22.000,8.1875,13.6364,9.2897,...,NaN,NaN,NaN,NaN,NaN,NaN,1.2144,0.1667,NaN,NaN
2,1991-02-28,21.291,5.1887,6.625,4.0625,22.0000,24.375,9.1875,14.8499,8.8234,...,NaN,NaN,NaN,NaN,NaN,NaN,1.3242,0.2292,NaN,NaN
3,1991-03-29,21.291,5.3705,6.625,5.1875,22.5625,22.500,9.6250,15.6588,8.5723,...,NaN,NaN,NaN,NaN,NaN,NaN,1.4047,0.2500,NaN,NaN
4,1991-04-30,21.441,5.6642,6.500,6.1250,23.3750,22.125,9.6875,14.4165,7.2452,...,NaN,NaN,NaN,NaN,NaN,NaN,1.4120,0.2986,NaN,NaN


### Skewness
In this step, we filter the dataset to include only the last six months of data, then calculate the percentage of missing values for each stock. We retain only those stocks with no more than 30% missing data (excluding the 'Dates' column). Next, we compute the skewness of returns for the selected stocks, sort them by skewness, and finally select the top 20% with the highest skewness values.

In [83]:
# Convert all columns except 'Dates' to numeric, coercing errors
for column in df_sp_d_new.columns:
    if column != 'Dates':
        df_sp_d_new[column] = pd.to_numeric(df_sp_d_new[column], errors='coerce')

# Convert 'Dates' column to datetime objects
df_sp_d_new['Dates'] = pd.to_datetime(df_sp_d_new['Dates'])

# Filter for the last 6 months
six_months_ago = df_sp_d_new['Dates'].max() - pd.DateOffset(months=6)
df_sp_d_last_6m_new = df_sp_d_new[df_sp_d_new['Dates'] >= six_months_ago]

# Calculate missing values percentage for each column
missing_percentage_new = df_sp_d_last_6m_new.isnull().mean() * 100

# Select columns with less than or equal to 30% missing values, excluding 'Dates'
cols_to_analyze_new = missing_percentage_new[missing_percentage_new <= 30].index.tolist()
if 'Dates' in cols_to_analyze_new:
    cols_to_analyze_new.remove('Dates')

# Calculate skewness for the selected columns
skewness_values_new = df_sp_d_last_6m_new[cols_to_analyze_new].skew()

# Sort skewness values and select the top 20%
top_20_percent_count_new = int(len(skewness_values_new) * 0.2)
top_skewness_new = skewness_values_new.sort_values(ascending=False).head(top_20_percent_count_new)

print("Top 20% titles with highest skewness in the last 6 months:")
print(top_skewness_new)

Top 20% titles with highest skewness in the last 6 months:
Keurig Dr Pepper Inc         1.377693
Edwards Lifesciences Corp    1.291127
Graham Holdings Co           1.289603
Big Lots Inc                 1.251414
McDonald's Corp              1.127863
                               ...   
Barnett Banks Inc            0.000000
BankBoston Corp              0.000000
BankAmerica Corp/Old         0.000000
Bank One Corp                0.000000
Bally Entertainment Corp     0.000000
Length: 270, dtype: float64


Scatterplot for the skewness for the entire S&P500, and scatterplot of the top 20% titles of S&P500 with highest skewness, in the last 6 months.

In [104]:
fig = px.scatter(
    x=skewness_values_new.index,
    y=skewness_values_new.values,
    labels={'x': 'Stock', 'y': 'Skewness'},
    title='Skewness of S&P 500 Stocks (Last 6 Months)'
)

pio.renderers.default = 'browser'
fig.show()

fig2 = px.scatter(
    x=top_skewness_new.index,
    y=top_skewness_new.values,
    labels={'x': 'Stock', 'y': 'Skewness'},
    title='Topo 20% Skewness of S&P 500 Stocks (Last 6 Months)'
)

fig2.show()

### Building the momentum portfolio
This section aims to construct a portfolio of 10 stocks based on a momentum investing strategy.
To compute momentum, we consider the price change over the past 12 months while excluding the most recent month to avoid the short-term reversal effect.
The momentum is computed with the formula: momentum_values = prices_1m_ago - prices_12m_ago.

In [ ]:
df = df_sp_d_new.copy()
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
df = df.sort_values(by=df.columns[0])

# Set the date column as an index to facilitate time slicing
df.set_index(df.columns[0], inplace=True)

# Define dates to compute momentum
last_date = df.index.max()
date_1m_ago = last_date - pd.DateOffset(months=1)
date_12m_ago = last_date - pd.DateOffset(months=12)

# Prices closest to 1 month and 12 months ago
prices_1m_ago = df.loc[:date_1m_ago].iloc[-1]
prices_12m_ago = df.loc[:date_12m_ago].iloc[-1]

# Computing momentum for each stock
momentum_values = prices_1m_ago - prices_12m_ago

# Dataframe with stocks and momentum values
momentum = pd.DataFrame({
    'titolo': momentum_values.index,
    'momentum': momentum_values.values
})

# Fixing top_skewness_new
titoli_skew = top_skewness_new.index.tolist()

# Filtering momentum for the titles with high skewness
momentum_filtered = momentum[momentum['titolo'].isin(titoli_skew)]

# Sorting by momentum and selecting the top 10
top10 = momentum_filtered.sort_values(by='momentum', ascending=False).head(10)

# Costruisci il portfolio prendendo i prezzi all'ultima data disponibile per i titoli top10
#portfolio = df.loc[last_date, top10['titolo']]

print("Top 10 Stocks with higher momentum:")
print(top10)

#print("\nPrezzi ultimi disponibili per il portfolio:")
#print(portfolio)


I 10 titoli con il momentum migliore:
                                 titolo  momentum
1159         Chipotle Mexican Grill Inc  157.3800
1015                      CME Group Inc  126.7250
36        Automatic Data Processing Inc  105.9307
1135               Booking Holdings Inc   96.2900
1077             Intuitive Surgical Inc   91.0400
684       Marriott International Inc/MD   71.6700
1029                IAC/InterActiveCorp   66.0100
1346  United Continental Holdings Inc.1   62.2200
975                        Intuit Inc.1   45.5800
810                          Intuit Inc   45.5800


In [103]:
# scatterplot dei titoli e il loro momentum
fig3 = px.scatter(
    x=momentum_filtered['titolo'],
    y=momentum_filtered['momentum'],
    labels={'x': 'Stock', 'y': 'Momentum'},
    title='Momentum of Selected S&P 500 Stocks with High Skewness (top20%, lsat 6 months)',
)
fig3.show()

### Comparison with the Markovitz optimal portfolio
Comparing a Markowitz mean-variance optimal portfolio with our momentum + skewness strategy is a great way to validate whether this factor-based method adds value. The S&P500 dataset will be used.